In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install seaborn

You should consider upgrading via the 'c:\users\ivbon\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



     -------------------------------------- 292.8/292.8 KB 2.3 MB/s eta 0:00:00


In [3]:
!c:\users\ivbon\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip
    

     ---------------------------------------- 2.1/2.1 MB 7.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.3
    Uninstalling pip-22.0.3:
      Successfully uninstalled pip-22.0.3
